In [1]:
# Install torch geometric
import os
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
  !pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1%2Bcu118.html


In [2]:
import torch_geometric
torch_geometric.__version__

'2.3.1'

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
from collections import defaultdict
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union
from torch_geometric.data import Data
import scipy.sparse
from torch import Tensor
from torch.utils.dlpack import from_dlpack, to_dlpack

from torch_geometric.utils.num_nodes import maybe_num_nodes

In [4]:
pos_edges=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/train.csv')

In [5]:
pos_edges = torch.tensor(pos_edges.values).T
num_nodes = pos_edges.max().item()
print(num_nodes)

1862220


In [6]:
def to_scipy_sparse_matrix(
    edge_index: Tensor,
    edge_attr: Optional[Tensor] = None,
    num_nodes: Optional[int] = None,
) -> scipy.sparse.coo_matrix:
    row, col = edge_index.cpu()

    if edge_attr is None:
        edge_attr = torch.ones(row.size(0))
    else:
        edge_attr = edge_attr.view(-1).cpu()
        assert edge_attr.size(0) == row.size(0)

    N = maybe_num_nodes(edge_index, num_nodes)
    out = scipy.sparse.coo_matrix(
        (edge_attr.numpy(), (row.numpy(), col.numpy())), (N, N))
    return out

In [8]:
adj_mat=to_scipy_sparse_matrix(pos_edges)

In [14]:
from torch_geometric.utils import negative_sampling
x = torch.rand(num_nodes+1, 16)
neg_edges=negative_sampling(pos_edges)

In [15]:
data = Data(x=x, edge_index=pos_edges)
data.neg_sampling=neg_edges

In [16]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class GNNStack(torch.nn.Module):                                                   #this makes a subclass of the nn.Module 
    def __init__(self, input_dim, hidden_dim, output_dim, args, emb=False):        #instance of the subclass(constructor method)
        super(GNNStack, self).__init__()                                           # initiates GNNstack instance 
        conv_model = self.build_conv_model(args.model_type)                        #building the conv layers (3 for us)         
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(args.heads * hidden_dim, hidden_dim))


        self.post_mp = nn.Sequential(                                                  #post processing dropout
            nn.Linear(args.heads * hidden_dim, hidden_dim), nn.Dropout(args.dropout),
            nn.Linear(hidden_dim, output_dim))

        self.dropout = args.dropout
        self.num_layers = args.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            return GAT
        
    def forward(self,data):
        x=data.x,pos_edges=data.edge_index
        for i in range(self.num_layers):
            x = self.convs[i](x,pos_edges)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout,training=self.training)
            
        x=self.post_mp(x)
        
        if self.emb==True:
            return x
        return F.log_softmax(x, dim=1)
    
    def get_node_embeddings(self, x):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=False)  # Set training to False to get deterministic output
        return x 
    
    def loss(self, pred, label):
        return F.nll_loss(pred, label) #negative log 